In [14]:
import pymongo
from flask import Flask
from pymongo.database import Database
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [15]:
def mongodb_init():
    # connect to mongodb
    mongo = pymongo.MongoClient(host='18.162.214.19', port=27017, username="root", password="1647#4hkust",
                                authSource='admin')
    print('数据库当前的databases: ', mongo.list_database_names())
    return mongo


def get_db(mongo, db_name):
    db = Database(name=db_name, client=mongo)
    print('获取/创建库：', db.name)
    return db


# API below
app = Flask(__name__)
mongo = mongodb_init()
db = get_db(mongo, 'chen_db')

数据库当前的databases:  ['admin', 'chen_db', 'config', 'local']
获取/创建库： chen_db


In [16]:
# data = {"PID":[],"Title":[],"Text":[]}
# Posts = db.Posts.aggregate([ {'$match': {'Deleted': {'$ne': True}}}])
# # Posts = db.Posts.find()
# if Posts:
#     for Post in Posts:
#         data["PID"].append(str(Post['_id']))
#         data["Title"].append(Post.get("Title").strip("\n"))
#         data["Text"].append(Post.get("Text").strip("\n"))
        
# df = pd.DataFrame(data)
# print(df)

In [17]:
data = []
data_Fields = {}
Posts = db.Posts.aggregate([ {'$match': {'Deleted': {'$ne': True}}} ])
# Posts = db.Posts.find()
if Posts:
    for Post in Posts:
        data.append([ str(Post['_id']), Post.get("Title").strip("\n"), Post.get("Text").strip("\n")])
        data_Fields[str(Post['_id'])] = Post.get("Fields")[0]
df = pd.DataFrame(data, columns=['PID', 'Title', 'Text'])
print(df["Text"])

In [ ]:
import re
import spacy
import unicodedata
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords

class Utils():
    def __init__(self):
        """ Class Constructor """
        self.stop_words = stopwords.words('english')
        unwanted_stopwords = {'no', 'nor', 'not', 'ain', 'aren', "aren't", 'couldn', 'what', 'which', 'who',
                                      'whom',
                                      'why', 'how', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't",
                                      'hasn',
                                      "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',
                                      "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn',
                                      "wasn't",
                                      'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't", 'don', "don't"}

        self.stop_words = [ele for ele in self.stop_words if ele not in unwanted_stopwords]
        self.wordnet_lemmatizer = WordNetLemmatizer()
        self.nouns = ['NNP', 'NNPS']
        self.nlp = spacy.load('en_core_web_sm')
        self.label_encoder = LabelEncoder()
        
    def clean_text(self, text, remove_stopwords=True, lemmatize=True):
            """ Function to clean text
            @param text (str): text to be cleaned
            @param remove_stopwords (bool): To remove stopwords or not.
            @param lemmatize (bool): to lemmatize or not.
            """

            # Remove emails 
            text = re.sub('\S*@\S*\s?', '', text)
            # Remove new line characters 
            text = re.sub('\s+', ' ', text) 
            # Remove distracting single quotes 
            text = re.sub("\'", '', text)
            # Remove puntuations and numbers
            text = re.sub('[^a-zA-Z]', ' ', text)
            # Remove single characters
            text = re.sub('\s+[a-zA-Z]\s+^I', ' ', text)
            # Remove accented words
            text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
            # remove multiple spaces
            text = re.sub(r'\s+', ' ', text)
            text = re.sub(r'^\s*|\s\s*', ' ', text).strip()
            text = text.lower()

            if not remove_stopwords and not lemmatize:
                return text

            # Remove unncecessay stopwords
            if remove_stopwords:
                text = word_tokenize(text)
                text = " ".join([word for word in text if word not in self.stop_words])
            
            # Word lemmatization
            if lemmatize:
                text = self.nlp(text)
                lemmatized_text = []
                for word in text:
                    if word.lemma_.isalpha():
                        if word.lemma_ != '-PRON-':
                            lemmatized_text.append(word.lemma_.lower())
                text = " ".join([word.lower() for word in lemmatized_text])
                    
            return text
        
    def prepare_data(self, df, remove_stopwords=True, lemmatize=True):
        df = df.sample(frac=1).reset_index(drop=True)
        x_train = df['Text'].apply(lambda x: self.clean_text(str(x), remove_stopwords=remove_stopwords, lemmatize=lemmatize))
        return x_train

In [ ]:
utils = Utils()

In [ ]:
prepare_data = utils.prepare_data(df, remove_stopwords=True, lemmatize=True)
print("Total training examples: ", len(prepare_data))

Total training examples:  18


In [ ]:
# 使用TF-IDF向量化商品标题和描述
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(prepare_data)
print(tfidf_matrix)

  (0, 9)	0.312528449208744
  (0, 22)	0.312528449208744
  (0, 58)	0.24590009751583847
  (0, 223)	0.312528449208744
  (0, 170)	0.312528449208744
  (0, 329)	0.625056898417488
  (0, 354)	0.312528449208744
  (0, 168)	0.24590009751583847
  (1, 213)	0.4756091253817318
  (1, 248)	0.33400436931487354
  (1, 64)	0.2920034154867396
  (1, 133)	0.39023733202794286
  (1, 340)	0.543372529978364
  (1, 346)	0.3597668200833718
  (2, 298)	0.38439180818422
  (2, 147)	0.4276194784385963
  (2, 283)	0.4885454577447716
  (2, 146)	0.4885454577447716
  (2, 64)	0.2625398495718696
  (2, 133)	0.3508618222056256
  (3, 332)	0.20230433523673638
  (3, 264)	0.23112806840059602
  (3, 360)	0.20230433523673638
  (3, 190)	0.23112806840059602
  (3, 187)	0.20230433523673638
  :	:
  (16, 115)	0.2793125385771961
  (16, 311)	0.2793125385771961
  (16, 119)	0.2793125385771961
  (16, 255)	0.2793125385771961
  (16, 312)	0.2793125385771961
  (16, 142)	0.5586250771543922
  (16, 184)	0.2793125385771961
  (16, 53)	0.24447977189082598
  

In [ ]:
from gensim.models import Word2Vec
import numpy as np

# 训练 Word2Vec 模型
model = Word2Vec(prepare_data, vector_size=100, window=5, min_count=1, sg=0)
# 获取单词的向量表示
word_vectors = model.wv

# 生成文档向量
document_vectors = []
for doc in prepare_data:
    doc_vector = np.mean([word_vectors[word] for word in doc], axis=0)
    document_vectors.append(doc_vector)
print(document_vectors)

[array([-0.08028915,  0.17125279,  0.2197987 ,  0.28753123, -0.02510177,
       -0.25112054,  0.22569776,  0.4854868 , -0.3349528 , -0.42525268,
        0.10753478, -0.33328378, -0.03618588,  0.14601417,  0.06687319,
        0.07470448,  0.34447843,  0.11895299, -0.15937014, -0.57320416,
        0.07937417,  0.0589574 ,  0.3102648 , -0.17717162, -0.04105014,
        0.18031752, -0.3322237 ,  0.17547199, -0.01552692,  0.09457127,
        0.2828334 , -0.19393875,  0.14662771, -0.20193934, -0.13583997,
        0.12068684,  0.24035506,  0.04336936, -0.05829236, -0.03369098,
        0.26253158, -0.13895027, -0.2158561 ,  0.19471018,  0.03079368,
       -0.06575144, -0.17927703, -0.13839677,  0.14444159,  0.11540981,
        0.07785697, -0.25464603, -0.0812418 , -0.00404877, -0.12377416,
        0.05464524,  0.14794913, -0.06905933,  0.11884388,  0.10173129,
       -0.16929041, -0.03598281,  0.3776003 ,  0.05368298, -0.19243218,
        0.45702118,  0.03253245,  0.16752325, -0.2692941 ,  0.0

In [ ]:
# 获取不同标签的数量
unique_values = set(data_Fields.values())
num_clusters = len(unique_values)

# 使用K-Means聚类算法将商品分为n个簇
kmeans = KMeans(n_clusters=num_clusters)
# kmeans.fit(tfidf_matrix)
kmeans.fit(document_vectors)

d:\Anaconda3\envs\ECE_651\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


KMeans(n_clusters=5)

In [ ]:
# 将聚类结果添加到数据集中
df['cluster'] = kmeans.labels_

In [13]:
# 打印每个簇中的商品
for cluster_id in range(num_clusters):
    cluster_products = df[df['cluster'] == cluster_id]['PID'].tolist()
    print(f"Cluster {cluster_id + 1}:")
    for product in cluster_products:
        print(f"  {product} with field {data_Fields[product]}")

Cluster 1:
  65cbc5334668f3bcc6b82323 with field Cameras & Camcorders
  65cbc53d4668f3bcc6b82325 with field Cars & Vehicles
  65cbc5494668f3bcc6b82328 with field Clothing
  65cbc54c4668f3bcc6b82329 with field Clothing
  65cbc5534668f3bcc6b8232b with field Bikes
  65cbc55a4668f3bcc6b8232d with field Musical Instruments
  65cbc55e4668f3bcc6b8232e with field Musical Instruments
  65cbc5624668f3bcc6b8232f with field Musical Instruments
  65cbc5654668f3bcc6b82330 with field Sporting Goods & Exercise
Cluster 2:
  65cbc5414668f3bcc6b82326 with field Cars & Vehicles
Cluster 3:
  65cbc52e4668f3bcc6b82322 with field Cameras & Camcorders
  65cbc5504668f3bcc6b8232a with field Furniture
  65cbc5734668f3bcc6b82333 with field Sporting Goods & Exercise
Cluster 4:
  65cbc5454668f3bcc6b82327 with field Clothing
  65cbc56a4668f3bcc6b82331 with field Sporting Goods & Exercise
  65cbc5ea4668f3bcc6b82334 with field Musical Instruments
Cluster 5:
  65cbc5384668f3bcc6b82324 with field Cars & Vehicles
  65cbc5